In [1]:
import tensorflow as tf

2022-08-01 08:58:07.472572: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-01 08:58:07.472635: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import requests
import pandas as pd
import numpy as np

In [3]:
global dfEntradas

In [13]:
url = "http://vps-40d69db1.vps.ovh.ca:23004/api/TreinoRedes/obtemListaPrevisaoTeste"

In [14]:
r = requests.get(url)
j = r.json()
dfTreino = pd.DataFrame.from_dict(j)

In [15]:
dfTreino.head()

,id,grupoAcaoId,periodoTreinoRedeId,tipoExemploTreinoId,regraProjecaoId,redeNeuralId,ativoPrevisao,limiteParaEntrada,tipoCompraVenda
0,2,12,1,1,30,2,1,0.5,V
1,8,12,1,1,32,2,1,0.3,C


In [5]:
idTreinoRede = 8
urlDias = "http://vps-40d69db1.vps.ovh.ca:23004/api/DiaPregaos/obtemPeriodoPrevisaoTeste?idPeriodoTreinoRede={}".format(idTreinoRede)


In [6]:
print(urlDias)

http://vps-40d69db1.vps.ovh.ca:23004/api/DiaPregaos/obtemPeriodoPrevisaoTeste?idPeriodoTreinoRede=8


In [ ]:
dia = 20220713
urlExemplos = "http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoEntradas/obtemPorDia?diaNum={}".format(dia)

In [ ]:
print(urlExemplos)

In [ ]:
global dfDados
r = requests.get(urlExemplos)
j = r.json()
dfDados = pd.DataFrame.from_dict(j)

In [ ]:
dfDados.head()

In [ ]:
def incluiEntrada(treino):
    urlEntrada = "http://vps-40d69db1.vps.ovh.ca:23004/api/CotacaoIntradayAcaoResultados/listaPrecoEntradaPrevisaoPorDiaB3?diaNum=" + str(dia) +"&idRegra=" + str(treino['regraProjecaoId']) + "&idGrupo=" + str(treino['grupoAcaoId'])
    print(urlEntrada)
    r = requests.get(urlEntrada)
    j = r.json()
    dfEntradas = pd.DataFrame.from_dict(j)
    print(dfEntradas.head())
    return dfEntradas

In [ ]:
def trataRede(treino):
    print('ID:' , treino['id'])
    nome_arquivo = 'pesos/treino_rede' + str(treino['id']) + '.h5'
    print(nome_arquivo)
    modelo = tf.keras.models.load_model(nome_arquivo)
    modelo.summary()
    result = modelo.predict(X)
    dfDados['resultado'] = result
    dfDados['treinoRedeId'] = str(treino['id'])
    dfDados['tipoCompraVenda'] = treino['tipoCompraVenda']
    dfEntradas = incluiEntrada(treino)
    dfPrevisao = pd.merge(dfDados,dfEntradas, how='left', left_on='ticker' ,  right_on='ticker')
    print('ticker previsão:' , len(dfPrevisao))
    json = dfPrevisao.to_dict(orient='list')
    urlPrevisao = 'http://vps-40d69db1.vps.ovh.ca:23004/api/PrevisaoRedes/recebePrevisaoTreino'
    print(json)
    response = requests.post(urlPrevisao, data=json)
    print(response)

In [ ]:
def trataLinhaX(linha):
    return np.fromstring(linha['campoX'], dtype=float, sep=',')

In [ ]:
dfTratadoX = dfDados.apply(trataLinhaX, axis='columns')
X = np.stack(dfTratadoX.values)
X.shape

In [ ]:
dfDados.drop('campoX', inplace=True, axis=1)

In [ ]:
dfTreino.apply(trataRede, axis='columns')